# Informações

In [1]:
import pandas as pd

from utils_ccd import get_connection,  get_info_file_path, extract_text_from_pdf, get_informacoes_processo

conn = get_connection()

In [ ]:
def get_info_df(processo):
    sql_all_informacoes_processos = '''
    SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
    ppe.SequencialProcessoEvento as evento,
    CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
    inf.*
    FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON inf.idinformacao = ppe.idinformacao
    WHERE concat(inf.numero_processo, '/', inf.ano_processo) = '{}'
    '''.format(processo)
    
    all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)
    all_informacoes_processos['caminho_arquivo'] = all_informacoes_processos.apply(get_info_file_path, axis=1)
    all_informacoes_processos['texto'] = all_informacoes_processos['caminho_arquivo'].apply(extract_text_from_pdf)

    return all_informacoes_processos

In [6]:
df_7486_2019 = get_info_df('007486/2019')

C:\Users\05911205424\AppData\Local\Temp\ipykernel_2084\728514825.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


In [8]:
df_7486_2019.head()

,arquivo,evento,processo,setor,numero_processo,ano_processo,ordem,data_resumo,resumo,sigilo,...,IdInformacaoSubstituida,UsuarioInformacaoSubstituida,DataInformacaoSubstituida,IdTipoParecer,Observacao,IdProcesso,E_Relatorio_Inicial,E_Relatorio_Auditoria,caminho_arquivo,texto
0,DAE_EXP_007486_2019_0055.pdf,84,007486/2019,DAE_EXP,007486,2019,55,2023-07-06 16:31:55.043,Processo Transitou em Julgado,0,...,None,None,None,None,None,NaN,None,None,\\10.24.0.6\tce$\Informacoes_PDF\DAE_EXP\DAE_E...,\nProcesso nº :\nInteressado :\n Diretoria de...
1,DE_007486_2019_0066.pdf,110,007486/2019,DE,007486,2019,66,2025-01-02 14:39:44.460,Processo_distribuicao_relator_TCE,None,...,None,None,None,None,None,NaN,None,None,\\10.24.0.6\tce$\Informacoes_PDF\DE\DE_007486_...,Processo nº :\nInteressado :\nAssunto :\nNo 2º...
2,DAE_EXP_007486_2019_0047.pdf,76,007486/2019,DAE_EXP,007486,2019,47,2023-03-16 13:48:40.847,Espelho de comunicação processual....,0,...,None,None,None,None,None,NaN,None,None,\\10.24.0.6\tce$\Informacoes_PDF\DAE_EXP\DAE_E...,Diretoria de Atos e Execução\nEmitido em:\n14/...
3,DAE_EXP_007486_2019_0050.pdf,79,007486/2019,DAE_EXP,007486,2019,50,2023-03-16 14:09:55.897,Motivo - mudança....,None,...,None,None,None,None,None,NaN,None,None,\\10.24.0.6\tce$\Informacoes_PDF\DAE_EXP\DAE_E...,"\nAv. Pres. GETÚLIO VARGAS, 690 – Ed. Dr. MÚC..."
4,DAM_FGE_007486_2019_0018.pdf,32,007486/2019,DAM_FGE,007486,2019,18,2021-08-30 09:36:26.353,Processo nº: 007486/2019 - TC\r\nJurisdicionad...,0,...,None,None,None,None,None,NaN,None,None,\\10.24.0.6\tce$\Informacoes_PDF\DAM_FGE\DAM_F...,"1 \nAvenida Presidente Getúlio Vargas, 690 – E..."


In [11]:
import shutil
import os
from PyPDF2 import PdfMerger

def download_pdfs_from_network(filepaths, dest_folder='pdfs_downloaded', final_filename='merged.pdf'):
    os.makedirs(dest_folder, exist_ok=True)
    for src_path in filepaths:
        filename = os.path.basename(src_path)
        dest_path = os.path.join(dest_folder, filename)
        try:
            shutil.copy(src_path, dest_path)
            print(f"Downloaded: {filename}")
        except Exception as e:
            print(f"Failed to download {filename}: {e}")
    def merge_pdfs(filepaths, output_path):
        merger = PdfMerger()
        for pdf in filepaths:
            try:
                merger.append(pdf)
            except Exception as e:
                print(f"Failed to append {pdf}: {e}")
        merger.write(output_path)
        merger.close()
        print(f"Merged PDF saved to {output_path}")

    # Merge all downloaded PDFs
    downloaded_folder = dest_folder
    downloaded_files = [os.path.join(downloaded_folder, os.path.basename(f)) for f in filepaths if os.path.exists(os.path.join(downloaded_folder, os.path.basename(f)))]
    merge_pdfs(downloaded_files, os.path.join(downloaded_folder, final_filename))

#download_pdfs_from_network(all_informacoes_processos['caminho_arquivo'], dest_folder='processos/017724_2017')
download_pdfs_from_network(df_7486_2019['caminho_arquivo'], dest_folder='processos/007486_2019', final_filename='007486_2019.pdf')


Downloaded: DAE_EXP_007486_2019_0055.pdf
Downloaded: DE_007486_2019_0066.pdf
Downloaded: DAE_EXP_007486_2019_0047.pdf
Downloaded: DAE_EXP_007486_2019_0050.pdf
Downloaded: DAM_FGE_007486_2019_0018.pdf
Downloaded: DAE_EXP_007486_2019_0030.pdf
Downloaded: DAM_007486_2019_0014.pdf
Downloaded: DAE_MANDA_007486_2019_0026.pdf
Downloaded: DAE_MANDA_007486_2019_0036.pdf
Downloaded: DAM_007486_2019_0004.pdf
Downloaded: DAE_EXP_007486_2019_0049.pdf
Downloaded: GCCTH_007486_2019_0012.pdf
Downloaded: DAE_MANDA_007486_2019_0041.pdf
Downloaded: DAE_MANDA_007486_2019_0058.pdf
Downloaded: GCCTH_007486_2019_0015.pdf
Downloaded: DAE_MANDA_007486_2019_0027.pdf
Downloaded: DE_007486_2019_0002.pdf
Downloaded: DAE_EXP_007486_2019_0032.pdf
Downloaded: DAE_EXP_007486_2019_0062.pdf
Downloaded: DAE_EXP_007486_2019_0035.pdf
Downloaded: GCCTH_007486_2019_0023.pdf
Downloaded: secpc_007486_2019_0024.pdf
Downloaded: DAE_EXP_007486_2019_0039.pdf
Downloaded: DE_007486_2019_0065.pdf
Downloaded: DAE_MANDA_007486_2019_000